In [1]:
# k crossover d，close > ema 8, ema 8 > ema 14, ema 14 > 50, take profit 2 atr, stop loss 3 atr

In [2]:
import websocket
import requests
import json
import pandas as pd
import datetime as dt
import numpy as np
import time
import ta
from dotenv import load_dotenv
import os
from binance.client import Client
from IPython.display import clear_output
import winsound

In [3]:
# 調整 notebook 寬物
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% ! important; }<style>"))

# 調整 output 不折疊
pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', None)

In [4]:
frequency = 250  # Set Frequency To 2500 Hertz
duration = 300  # Set Duration To 1000 ms == 1 second
winsound.Beep(frequency, duration)

In [5]:
load_dotenv()

api_key = os.getenv('API_KEY')
api_secret = os.getenv('SECRET_KEY')

client = Client(api_key, api_secret, testnet = False)

spot = client.get_account()
s_bal = pd.DataFrame(spot['balances'])
print(s_bal)


    asset        free      locked
0     BTC  0.00000000  0.00000000
1     LTC  0.00000000  0.00000000
2     ETH  0.00000000  0.00000000
3     NEO  0.00000000  0.00000000
4     BNB  0.04622450  0.00000000
..    ...         ...         ...
530   SYN  0.00000000  0.00000000
531  LQTY  0.00000000  0.00000000
532    ID  0.00000000  0.00000000
533   ARB  0.00000000  0.00000000
534  RDNT  0.00000000  0.00000000

[535 rows x 3 columns]


In [6]:
timezone = 8
endpoint = 'wss://stream.binance.com:9443/ws'
symbol = 'ethusdt'
symbol_C = symbol.upper()
interval = '15m'

# start epoch till now
start_time = 1679533448000
end_time = round(time.time() * 1000)

# step between timestamps in milliseconds, 60000 = 1min 
step = 60000 * 3600

In [7]:
data = json.dumps({'method':'SUBSCRIBE','params':[symbol + '@kline_' + interval],'id':1})

In [8]:
# 先抓歷史資料好讓技術指標能成型
def get_historical(symbol, interval, start_time, end_time, step):
    
    rawdf = pd.DataFrame()
    
    url = "https://api.binance.com/api/v3/klines"
    
    for timestamp in range(start_time, end_time, step):
        params = {"symbol": symbol_C,
                  "interval": interval,
                  "startTime": timestamp,
                  "endTime": timestamp + step}
        response = requests.get(url, params=params).json()
        out = pd.DataFrame(response, columns = ["Open time", "Open", "High", "Low", "Close",
                                               "Volume", "Close_Time", "Quote asset volume",
                                               "Number of trades", "Taker buy base asset volume",
                                               "Taker buy quote asset volume", "Ignore"])
        rawdf = pd.concat([rawdf, out], axis = 0)
    
    rawdf = rawdf[['Close_Time', 'Open', 'Close', "High", "Low", 'Volume']]
    convert_dict = {'Close_Time': float, 'Open': float, 'Close': float, "High": float, "Low": float, 'Volume': float}
    rawdf = rawdf.astype(convert_dict)

    rawdf['Close_Time'] = pd.to_datetime(rawdf['Close_Time'], unit = 'ms')
    rawdf['Close_Time'] = rawdf['Close_Time'] + pd.Timedelta(hours=timezone)
    rawdf['Close_Time'] = rawdf['Close_Time'].dt.strftime('%Y-%m-%d %H:%M:%S')
    
    rawdf = rawdf.reset_index(drop=True)
    
    return rawdf

In [9]:
rawdf = get_historical(symbol, interval, start_time, end_time, step)
print(rawdf)

              Close_Time     Open    Close     High      Low      Volume
0    2023-03-23 09:29:59  1734.35  1733.60  1736.16  1731.41   3640.5464
1    2023-03-23 09:44:59  1733.61  1732.90  1735.89  1732.58   3048.7296
2    2023-03-23 09:59:59  1732.91  1730.06  1734.00  1729.54   4409.5993
3    2023-03-23 10:14:59  1730.06  1734.54  1734.82  1729.35   2969.2861
4    2023-03-23 10:29:59  1734.54  1735.44  1735.93  1733.92   2705.6078
..                   ...      ...      ...      ...      ...         ...
819  2023-03-31 23:29:59  1837.23  1828.75  1838.01  1827.73  17854.6508
820  2023-03-31 23:44:59  1828.75  1827.94  1832.60  1824.06   8344.0454
821  2023-03-31 23:59:59  1827.95  1830.60  1830.60  1826.22   4853.9329
822  2023-04-01 00:14:59  1830.60  1830.11  1834.64  1829.20   4521.0381
823  2023-04-01 00:29:59  1830.11  1829.82  1831.02  1829.81    358.7942

[824 rows x 6 columns]


In [10]:
def indicators(df):

# ema
    for i in (8, 18, 38):
        df['ema_'+str(i)] = ta.trend.ema_indicator(df.Close, window=i)

# atr
    df['atr'] = ta.volatility.average_true_range(df.High, df.Low, df.Close)

# kd
    kd_int = 14
    d_int = 3
   
    kddf = pd.DataFrame()
    kddf[str(kd_int) + '-Low'] = df['Low'].rolling(kd_int).min()
    kddf[str(kd_int) + '-High'] = df['High'].rolling(kd_int).max()
    df['slowk'] = (df['Close'] - kddf[str(kd_int) + '-Low'])*100/(kddf[str(kd_int) + '-High'] - kddf[str(kd_int) + '-Low'])
    df['slowd'] = df['slowk'].rolling(d_int).mean()
    
    return df

def checkcross(df):
    series = df['slowk'] < df['slowd']
    return series.diff()

In [11]:
def conditions(df):
    # k crossover d，close > ema 8, ema 8 > ema 14, ema 14 > 50, take profit 2 atr, stop loss 3 atr
    df['kd_cross'] = checkcross(df)

    for index, row in df.iterrows():
        # c1
        df['c1'] = df['kd_cross']
        # c2
        df['c2'] = df['Close'] <= df['ema_8']
        # c3
        df['c3'] = df['ema_8'] <= df['ema_18']
        # c4
        df['c4'] = df['ema_18'] <= df['ema_38']

    # 條件達成
    df['signal'] = False
    df.loc[df.c1 & df.c2 & df.c3 & df.c4, 'signal'] = True
            
    # 下一根進場
    df['open_entry'] = False

    return df

In [12]:
def enter_position(df):
    global symbol_C
    
    #-----Calculate entry price-----#
    close_val = df['Close']
    atr_val = df['atr']
    df.loc[df.index[-1], 'entry_p'] = close_val.loc[close_val.index[-2]]
    df.loc[df.index[-1], 'stop_loss'] = close_val.loc[close_val.index[-2]] + 3 * atr_val.loc[atr_val.index[-2]]
    df.loc[df.index[-1], 'take_profit'] = close_val.loc[close_val.index[-2]] - 7 * atr_val.loc[atr_val.index[-2]]

    #-----position attributes-----#
    quantity = 0.1
    entry_p = df.loc[df.index[-2], 'Close']
    stop_loss_p = round(df.loc[df.index[-1], 'stop_loss'],2)
    take_profit_p = round(df.loc[df.index[-1], 'take_profit'],2)

    print(df.loc[df.index[-1], 'entry_p'] == entry_p)  
    
    open_orders = client.futures_get_open_orders()
    # Cancel all open orders
    if open_orders:
        # Cancel all open orders
        for order in open_orders:
            cancel_response = client.futures_cancel_order(symbol=order['symbol'], orderId=order['orderId'])
            print(f"Canceled order: {order['symbol']} - {order['orderId']}")
    else:
        # Continue with the rest of the code
        print("No open orders found.")
        time.sleep(1)  # Sleep for 1 second to avoid API rate limit
    
    try:
        order = client.futures_create_order(
            symbol=symbol_C,
            side='SELL',
            type='LIMIT',
            timeInForce='GTC',
            quantity=quantity,
            price=entry_p,
            positionSide='SHORT',
        )
        print('Order created successfully.')       
        
        stop_loss_order = client.futures_create_order(
            symbol=symbol_C,
            side='BUY',
            type='STOP_MARKET',
            stopPrice=stop_loss_p,
            stopLimitTimeInForce='GTC',
            closePosition = 'true',
            positionSide = 'SHORT',
        )
        print('Stop loss created successfully')
        
        take_profit_order = client.futures_create_order(
            symbol=symbol_C,
            side='BUY',
            type='TAKE_PROFIT_MARKET',
            stopPrice=take_profit_p,
            stopLimitTimeInForce='GTC',
            closePosition = 'true',
            positionSide = 'SHORT',
        )
        winsound.Beep(frequency, duration)
        print('Take profit created successfully')
            
    except Exception as e:
        print(f'Error creating order: {e}')

    print(str(symbol_C) + ' entered at ' + str(entry_p) + ', stop loss at ' + str(stop_loss_p))


In [13]:
line_count = 0
max_lines = 3

current_k = 0

def on_open(ws):
    ws.send(data)
    
def on_message(ws, message):
    global current_k, line_count, max_lines
    df = get_historical(symbol, interval, start_time, end_time, step)

    indicators(df)
    conditions(df)
    
    try:
        # 如果時間標籤改變，代表上個 Close 發生，檢查是否進場
        if df['Close_Time'][len(df) - 1] != current_k:
            print('time changed')
            current_k = df['Close_Time'][len(df) - 1]
            if df.iloc[df.index[-2]]['signal'] == True:
                winsound.Beep(frequency, duration)
                df.loc[df.index[-1], 'open_entry'] = True
                enter_position(df)
                print(df.tail(2))
        
        # 隨時偵測出場條件是否成立，不必等 Close 發生
#         if ((df.loc[df.index[-1], 'rsi'] >= tp_rsi) & (df.loc[df.index[-1], 'slowk'] >= tp_slowk) & (df.loc[df.index[-1], 'slowd'] >= tp_slowd)):
#             take_profit_order = client.futures_create_order(
#                 symbol=symbol_C,
#                 side='SELL',
#                 type='TAKE_PROFIT_MARKET',
#                 stopPrice=df.loc[df.index[-1], 'Close'],
#                 stopLimitTimeInForce='GTC',
#                 closePosition = 'true',
#                 positionSide = 'LONG'
#             )
#             winsound.Beep(frequency, duration)
#             print('Take profit created successfully')
        
    except Exception as e:
        print(f'Error creating order: {e}')


    try:
        df = df.reset_index(drop=True)
        print('nothing happened...' + str(df.loc[df.index[-1], 'Close_Time']) + ' ' + str(df.loc[df.index[-1], 'Close']))
        print()
        print(df.tail(2))
        print('----------------------------------------------------------------------------')
        
        positions = client.futures_account()['positions']
        for position in positions:
            if float(position['positionAmt']) != 0:
                position_df = pd.DataFrame({'Symbol':position['symbol'],
                                            'Entry_P':round(float(position['entryPrice']),2),
                                            'Amt':round(float(position['positionAmt']) * df.loc[df.index[-1], 'Close'],2),
                                            'PL':round(float(position['unrealizedProfit']),2),
                                            'X':round(float(position['leverage']),1),
                                           }, index=[0])      
                print(position_df)
                print()
    
    except Exception as e:
        print(f'Error: {e}')    
        
        
    line_count += 1
    if line_count >= max_lines:
        clear_output(wait=True)
        line_count = 0

In [ ]:
ws = websocket.WebSocketApp(endpoint, on_message = on_message, on_open = on_open)
ws.run_forever()

nothing happened...2023-04-01 12:14:59 1826.5

              Close_Time     Open    Close     High      Low     Volume        ema_8       ema_18       ema_38       atr      slowk      slowd kd_cross     c1    c2     c3     c4  signal  open_entry
869  2023-04-01 11:59:59  1826.01  1825.99  1827.50  1825.65  1645.5064  1827.117384  1826.665688  1824.934491  4.536624  27.476314  28.064887    False  False  True  False  False   False       False
870  2023-04-01 12:14:59  1825.99  1826.50  1827.05  1824.40  1723.6272  1826.980187  1826.648247  1825.014774  4.401865  28.471310  27.851045     True   True  True  False  False   False       False
----------------------------------------------------------------------------
    Symbol  Entry_P    Amt    PL     X
0  ETHUSDT  1828.93  365.3 -0.56  10.0

    Symbol  Entry_P     Amt    PL     X
0  ETHUSDT   1809.0 -301.37 -2.82  10.0

nothing happened...2023-04-01 12:14:59 1826.49

              Close_Time     Open    Close     High      Low     Volume